In [69]:
from google.cloud import bigquery
import pandas as pd

In [80]:
base_df = pd.read_csv(r'C:\Users\VinodKumarM\OneDrive - EQUINIX\Projects\fusion_automate_project\resources\BASE_DF.csv')

In [81]:
module_list = list(base_df.MODULE.unique())
module_list = "\""+'","'.join(module_list)+"\""

client = bigquery.Client('helix-data-dev')
sql = "select MODULE, max(table_id) MAX_TABLE_ID from helix-data-dev.BQ_CTL_METADATA.FUSION_TO_BQ_PVO_DTL \
WHERE MODULE IN ({0}) AND SOURCE_TYPE='BICC' GROUP BY MODULE ".format( module_list)

print(sql)
pvodf = client.query(sql).to_dataframe()
pvodict = pvodf.to_dict('records')

max_tableids = dict()
for a in pvodict:
    max_tableids[a['MODULE']] = int(a['MAX_TABLE_ID'].split('_')[-1])

select MODULE, max(table_id) MAX_TABLE_ID from helix-data-dev.BQ_CTL_METADATA.FUSION_TO_BQ_PVO_DTL WHERE MODULE IN ("MS","AR","S2P") AND SOURCE_TYPE='BICC' GROUP BY MODULE 


In [82]:
def derive_table_id(x):
    table_id = max_tableids[x.MODULE]
    table_id = table_id+1
    max_tableids[x.MODULE] = table_id
    return x.MODULE+'_'+str(table_id).rjust(3,'0')

In [83]:
base_df['TABLE_ID'] = base_df.apply(lambda x:derive_table_id(x),axis=1)

In [84]:
base_df

,DATASTORE_NAME,SOURCE_TABLE_NAME,MODULE,TABLE_ID
0,FscmTopModelAM.PartiesAnalyticsAM.OriginalSyst...,HZ_ORIG_SYS_REFERENCES,MS,MS_030
1,FscmTopModelAM.PartiesAnalyticsAM.OriginalSyst...,HZ_ORIG_SYS_REFERENCES,MS,MS_031
2,FscmTopModelAM.PartiesAnalyticsAM.OriginalSyst...,HZ_ORIG_SYS_REFERENCES,MS,MS_032
3,FscmTopModelAM.PartiesAnalyticsAM.OriginalSyst...,HZ_ORIG_SYS_REFERENCES,MS,MS_033
4,FscmTopModelAM.PartiesAnalyticsAM.OriginalSyst...,HZ_ORIG_SYS_REFERENCES,MS,MS_034
5,FscmTopModelAM.PartiesAnalyticsAM.OriginalSyst...,HZ_ORIG_SYS_REFERENCES23,AR,AR_047
6,FscmTopModelAM.PartiesAnalyticsAM.OriginalSyst...,HZ_ORIG_SYS_REFERENCES23,AR,AR_048
7,FscmTopModelAM.PartiesAnalyticsAM.OriginalSyst...,HZ_ORIG_SYS_REFERENCES23,S2P,S2P_031
8,FscmTopModelAM.PartiesAnalyticsAM.OriginalSyst...,HZ_ORIG_SYS_REFERENCES23,S2P,S2P_032
